## Основные характеристики датасета

MedInc - медианный доход в районе

HouseAge - средний возраст домов в районе

AveRooms - среднее количество комнат на дом

AveBedrms - среднее количество спален на дом

Population - население района

AveOccup - среднее количество жителей на дом

Latitude - географическая широта района

Longitude - географическая долгота района

MedHouseVal - медианная стоимость домов в районе (целевая переменная)

## Подготовка

In [ ]:
!pip install gmdh

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
import gmdh

RANDOM_STATE=123

import warnings
warnings.simplefilter('ignore')

In [ ]:
# Загрузка данных
california = fetch_california_housing()
data = pd.DataFrame(data= np.c_[california['data'], california['target']],
                     columns= california['feature_names'] + ['target'])

In [ ]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
data.isnull().sum()

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
target        0
dtype: int64

In [ ]:
data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


## Разделение на выборки

In [ ]:
X = data.iloc[:, :-1]
y = data.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Обучение моделей

In [ ]:
# Стекинг

estimators = [
    ('lr', LinearRegression()),
    ('dt', DecisionTreeRegressor(random_state=RANDOM_STATE))
]

stacking = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking.fit(X_train_scaled, y_train)

StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('dt', DecisionTreeRegressor(random_state=123))],
                  final_estimator=LinearRegression())

In [ ]:
# Модель многослойного персептрона

mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=RANDOM_STATE)
mlp.fit(X_train_scaled, y_train)

MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=123)

In [ ]:
# Линейный метод COMBI

combi = gmdh.Combi()
combi.fit(X_train_scaled, y_train)
combi.get_best_polynomial()

'y = 0.8542*x1 + 0.1137*x2 - 0.321*x3 + 0.3143*x4 - 0.8748*x7 - 0.8305*x8 + 2.0719'

In [ ]:
# Нелинейный метод MIA

mia = gmdh.Mia()
mia.fit(X_train_scaled, y_train)


## Оценка моделей

In [ ]:
y_pred_stacking = stacking.predict(X_test_scaled)
y_pred_mlp = mlp.predict(X_test_scaled)
y_pred_combi = combi.predict(X_test_scaled)
y_pred_mia = mia.predict(X_test_scaled)

In [ ]:
# MAE
print(f"Stacking: {mean_absolute_error(y_test, y_pred_stacking):.4f}")
print(f"Perceptron: {mean_absolute_error(y_test, y_pred_mlp):.4f}")
print(f"COMBI: {mean_absolute_error(y_test, y_pred_combi):.4f}")
print(f"MIA: {mean_absolute_error(y_test, y_pred_mia):.4f}")

Stacking: 0.4291
Perceptron: 0.3503
COMBI: 0.5289
MIA: 0.5820


In [ ]:
# R^2
print(f"Stacking: {r2_score(y_test, y_pred_stacking):.4f}")
print(f"Perceptron: {r2_score(y_test, y_pred_mlp):.4f}")
print(f"COMBI: {r2_score(y_test, y_pred_combi):.4f}")
print(f"MIA: {r2_score(y_test, y_pred_mia):.4f}")

Stacking: 0.7083
Perceptron: 0.8088
COMBI: 0.6058
MIA: 0.5343


## Вывод

Многослойный перцептрон показал наилучшие результаты среди всех моделей как по MAE, так и по R² для данной задачи. Однако и на его обучение ушло больше всего времени.

Стекинг модель с простой архитектурой оказалась на втором месте после персептрона. Если подобрать другие модели и параметры для них она могла бы показать себя лучше.

COMBI и MIA показали результаты хуже. При этом нелинейный метод, способный улавливать более сложные зависимости показал себя хуже линейного.